<a href="https://colab.research.google.com/github/ppprakharr/ClassificationModels/blob/main/MRI_Scan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import os
import json

In [9]:
kaggle_credentials = json.load(open("kaggle.json"))
os.environ['KAGGLE_USERNAME']=kaggle_credentials['username']
os.environ['KAGGLE_KEY']=kaggle_credentials['key']

In [10]:
!kaggle datasets download -d masoudnickparvar/brain-tumor-mri-dataset


100% 148M/149M [00:07<00:00, 22.0MB/s]
100% 149M/149M [00:07<00:00, 20.6MB/s]


In [11]:
#unzipping
zip_path='/content/brain-tumor-mri-dataset.zip'
from zipfile import ZipFile
with ZipFile(zip_path,'r') as zip:
  zip.extractall()


In [12]:
import random
random.seed(1)

import numpy as np
np.random.seed(1)

import tensorflow as tf
tf.random.set_seed(1)

In [75]:
!ls


brain-tumor-mri-dataset.zip  kaggle.json  sample_data  Testing	Training


In [13]:
import json
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers,models

In [14]:
print(len(os.listdir('/content/Training')))

4


In [15]:
print(os.listdir('/content/Training/')[0:5])

['pituitary', 'meningioma', 'glioma', 'notumor']


In [16]:
print(len(os.listdir('/content/Training/notumor')))

1595


In [17]:
print(os.listdir('/content/Training/notumor')[0:5])

['Tr-no_0937.jpg', 'Tr-no_0870.jpg', 'Tr-no_0852.jpg', 'Tr-no_1555.jpg', 'Tr-no_1327.jpg']


In [18]:
#base dir
base_dir = '/content/Training/'

In [19]:
# image data generator
data_gen=ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

In [20]:
img_size=224
batch_size=32

In [21]:
#train generator
train_generator = data_gen.flow_from_directory(
    base_dir,
    target_size=(img_size,img_size),
    batch_size=batch_size,
    subset='training',
    class_mode='categorical'
)

Found 4571 images belonging to 4 classes.


In [22]:
#test generator
validation_generator = data_gen.flow_from_directory(
    base_dir,
    target_size=(img_size,img_size),
    batch_size=batch_size,
    subset='validation',
    class_mode='categorical'
)

Found 1141 images belonging to 4 classes.


In [23]:
model=models.Sequential()
model.add(layers.Conv2D(32,(3,3),activation='relu',input_shape=(img_size,img_size,3)))
model.add(layers.MaxPooling2D(2,2))
model.add(layers.Dropout(0.5))
model.add(layers.Conv2D(64,(3,3),activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.MaxPooling2D(2,2))

model.add(layers.Flatten())
model.add(layers.Dense(256,activation='relu'))
model.add(layers.Dense(train_generator.num_classes,activation='softmax'))

In [87]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 111, 111, 32)      0         
 g2D)                                                            
                                                                 
 dropout_2 (Dropout)         (None, 111, 111, 32)      0         
                                                                 
 conv2d_5 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 dropout_3 (Dropout)         (None, 109, 109, 64)      0         
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 54, 54, 64)        0         
 g2D)                                                 

In [24]:
# compile the model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [27]:
# training the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples//batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples//batch_size,
)

Epoch 1/10
142/142 [==============================] - 23s 117ms/step - loss: 1.8668 - accuracy: 0.5739 - val_loss: 1.0500 - val_accuracy: 0.6125
Epoch 2/10
142/142 [==============================] - 16s 109ms/step - loss: 0.4701 - accuracy: 0.8244 - val_loss: 0.9192 - val_accuracy: 0.6384
Epoch 3/10
142/142 [==============================] - 16s 110ms/step - loss: 0.2534 - accuracy: 0.9039 - val_loss: 0.7502 - val_accuracy: 0.7268
Epoch 4/10
142/142 [==============================] - 16s 110ms/step - loss: 0.1400 - accuracy: 0.9511 - val_loss: 0.8157 - val_accuracy: 0.7161
Epoch 5/10
142/142 [==============================] - 16s 109ms/step - loss: 0.0739 - accuracy: 0.9731 - val_loss: 0.8995 - val_accuracy: 0.7509
Epoch 6/10
142/142 [==============================] - 16s 111ms/step - loss: 0.0501 - accuracy: 0.9841 - val_loss: 1.0029 - val_accuracy: 0.7750
Epoch 7/10
142/142 [==============================] - 15s 108ms/step - loss: 0.0333 - accuracy: 0.9896 - val_loss: 1.0746 - val_ac

In [28]:
import pickle

In [29]:
filename='mriScanModel.sav'
pickle.dump(model,open(filename,'wb'))
mri = pickle.load(open(filename,'rb'))